#Задание 13
Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser решить задачу парафраза

(дополнительно необязательная задача)на выбор взять https://huggingface.co/datasets/sberquad https://huggingface.co/datasets/blinoff/medical_qa_ru_data натренировать любую модель для вопросно ответной системы как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

In [38]:
!pip install transformers sentencepiece --quiet    
!pip install datasets --quiet     

In [39]:
import pandas as pd

#Загружаем датасет

In [40]:
from datasets import load_dataset
corpus = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')

  0%|          | 0/1 [00:00<?, ?it/s]

In [41]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [42]:
corpus['train'][0:4]

{'id': ['25349', '25360', '25382', '25423'],
 'id_1': ['34420', '34438', '34475', '34494'],
 'id_2': ['34421', '34439', '34476', '34519'],
 'text_1': ['Цены на нефть восстанавливаются',
  '"Гоголь-центр" покажет видеозапись скандального спектакля "Тангейзер"',
  'Агент: РФС вновь задерживает зарплату Фабио Капелло',
  'День Победы в Москве обещает выдаться облачным'],
 'text_2': ['Парламент Словакии поблагодарил народы бывшего СССР за Победу',
  'Кехман запретил «Гоголь-центру» показывать видеозапись «Тангейзера»',
  'СМИ: Агент Фабио Капелло грозится подать в суд на РФС',
  'Любляна отпразднует День Победы вместе с Москвой'],
 'class': ['-1', '-1', '-1', '-1']}

In [43]:
print(corpus['train']['text_1'][1])
print(corpus['train']['text_2'][1])     

"Гоголь-центр" покажет видеозапись скандального спектакля "Тангейзер"
Кехман запретил «Гоголь-центру» показывать видеозапись «Тангейзера»


#Используем предобученную модель 'ruRoberta-large-paraphrase-v1'


In [44]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained('SkolkovoInstitute/ruRoberta-large-paraphrase-v1')
tokenizer = AutoTokenizer.from_pretrained('SkolkovoInstitute/ruRoberta-large-paraphrase-v1')

def get_similarity(text1, text2):
    """ Predict the probability that two Russian sentences are paraphrases of each other. """
    with torch.inference_mode():
        batch = tokenizer(
            text1, text2, 
            truncation=True, max_length=model.config.max_position_embeddings, return_tensors='pt',
        ).to(model.device)
        proba = torch.softmax(model(**batch).logits, -1)
    return proba[0][1].item()

print(get_similarity('Я тебя люблю', 'Ты мне нравишься'))  # 0.9798
print(get_similarity('Я тебя люблю', 'Я тебя ненавижу'))   # 0.0008

0.9798762798309326
0.0008293354767374694


In [52]:
result_df = []

def paraphrase_check(row_numb):
    
    text1 = corpus['train']['text_1'][row_numb]
    text2 = corpus['train']['text_2'][row_numb]
    
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).numpy()
    
    p = proba[0]
    if p[1] > 0.5:
        paraphrase = "Парафраз"
    else:
        paraphrase = "Не парафраз"
    
    result_df.append([text1, text2, p[1], paraphrase])
    
    return pd.DataFrame(result_df, columns=['текст 1', 'текст 2', 'вероятность парафраза', 'Результат'])

In [53]:
pd.set_option('display.max_colwidth', None)

In [54]:
paraphrase_check(1)
paraphrase_check(2)
paraphrase_check(3)
paraphrase_check(4)
paraphrase_check(5)
paraphrase_check(6)
paraphrase_check(7)

,текст 1,текст 2,вероятность парафраза,Результат
0,"""Гоголь-центр"" покажет видеозапись скандального спектакля ""Тангейзер""",Кехман запретил «Гоголь-центру» показывать видеозапись «Тангейзера»,0.004268,Не парафраз
1,Агент: РФС вновь задерживает зарплату Фабио Капелло,СМИ: Агент Фабио Капелло грозится подать в суд на РФС,0.000674,Не парафраз
2,День Победы в Москве обещает выдаться облачным,Любляна отпразднует День Победы вместе с Москвой,0.000839,Не парафраз
3,Посол РФ в США: Россия будет бороться с попытками переписать историю,Правительство запланировало заработать на лотереях 400 млрд руб.,0.000759,Не парафраз
4,Вертолет с 11 иностранцами на борту упал в Пакистане,В Пакистане упал вертолет с 11 иностранцами,0.991622,Парафраз
5,Самолет вернулся в аэропорт Новосибирска из-за стука в салоне,Самолет вернулся в новосибирский аэропорт из-за таинственного стука,0.629243,Парафраз
6,Васильева признана виновной в мошенничестве и хищениях,Васильева признана виновной в хищениях и отмывании денег,0.000704,Не парафраз


**ВЫВОД: с задачей классификации парафраза модель справляется успешно.**